In [ ]:
%load_ext autoreload
%autoreload 2
from helper import *
pd.options.plotting.backend = "plotly"

import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Load data
- Loads the forecated data 

In [ ]:
no1_data = pd.read_csv('./data/time_series_60min_singleindex_filtered.csv', usecols=['timestamp', 'NO_1_price_day_ahead', 'NO_1_load_actual_entsoe_transparency', 'NO_1_load_forecast_entsoe_transparency']).fillna(0)
price = no1_data['NO_1_price_day_ahead'] 
price[:160].plot()

## Model formulation

$$
\begin{gather}
    \begin{aligned}
    J_N^*(x_0) := \quad  \max_{\textbf{u}}&\quad J_N(x_0, \textbf{u}), &\\
        \mathrm{s.t.} \quad  
                    E(t+1) &= E(t) + \eta_c P_c(t) - \frac{P_d(t)}{\eta_d}, &&t =0,1,\ldots,N-1\\
            \underline{E} &\leq E(t) \leq \overline{E}, && t=0,1,\ldots,N\\
                        0   &\leq P_c(t) \leq \overline{P_c}, && t=0,1,\ldots,N\\
                        0 &\leq P_d(t) \leq \overline{P_d}, && t=0,1,\ldots,N \\
                            &  E(0) = x_0
    \end{aligned}
\end{gather}
$$

In [ ]:
def simple_model(price_data, horizon_length):
    m = pyo.ConcreteModel("Energy storage model deterministic")
    # Number of timesteps in planning horizon
    m.top = pyo.RangeSet(0, horizon_length - 1)
    m.N = pyo.Set(within=m.top, initialize=range(horizon_length))

    # Square root of round trip efficiency
    m.sqrteta_c = pyo.Param(initialize = np.sqrt(0.90))
    m.sqrteta_d = pyo.Param(initialize = np.sqrt(0.96))
    m.pPrice = pyo.Param(m.N, initialize = price_data[:horizon_length], domain=pyo.NonNegativeReals, mutable=True)
    m.E0 = pyo.Param(initialize=0, mutable=True, doc="Initial state of battery")

    m.vChargeRate = pyo.Var(m.N, initialize = 0.0, bounds=(0.0, 2), doc="Charging rate [MWh]")
    m.vDischargeRate = pyo.Var(m.N, initialize = 0.0, bounds=(0, 4), doc="Discharge rate [MWh]")
    m.E = pyo.Var(m.N, initialize = 0.0, bounds=(0, 6), doc="State of charge [MWh]")

    @m.Expression(m.N, doc="Charging and discharging expression at time t")
    def P_util(model,t):
        return model.vChargeRate[t]*model.sqrteta_c - (model.vDischargeRate[t] / model.sqrteta_d)

    # This expression is for plotting the data
    @m.Expression(m.N, doc="Cumulative sum of profits at time t")
    def netsum(model, t):
        return model.pPrice[t] * (model.vDischargeRate[t] - model.vChargeRate[t])

    # Note: this model assumes 1-hour timestep in price data and control actions.
    @m.Constraint(m.N, doc="Energy preservation constraint")
    def _energyBalance(model,t):
        if t == 0 :
            return model.E[t] == model.E0
        else :
            return model.E[t] == model.E[t-1] + model.P_util[t]

    @m.Objective(sense=pyo.minimize)
    def _objfunLP(model):
        return sum((model.vChargeRate[t] - model.vDischargeRate[t]) * model.pPrice[t] for t in model.N)

    return m

## Simulate one iteration solve for a length of 150
- The model is solved over one iteration
- THe model only relies on the forecast data from NO1 bidding zone

In [ ]:
def solve_system_deterministic(price, h_length=40 ,initial_bess_level=2.0, tee=False):
    """Solve energy system deterministically"""
    m_energy = simple_model(price,horizon_length=h_length)

    # Specify the solver
    solver = pyo.SolverFactory('gurobi')

    # Fix the initial energy amount
    m_energy.E0 = initial_bess_level
    m_energy.vChargeRate[0].fix()
    m_energy.vDischargeRate[0].fix()

    results = solver.solve(m_energy, tee=tee)
    return m_energy, results

def plot_simple_model_deterministic(m_energy):
    """Plot the deterministic energy model """
    df = storage_extract_singleindex(m_energy)

    fig = make_subplots(rows=2, cols=2, subplot_titles=("Price", "Charging Rate", "Cumulative Profit", "Battery Charge"))
    fig.add_trace(
        go.Scatter(x=df.index, y=df['pPrice'], name='Price', mode='lines'),
        row=1, col=1)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['vChargeRate'], name='Charging Rate', line={'shape':'hv'}),
        row=1, col=2)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['vDischargeRate'], name='Discharging Rate', line={'shape':'hv'}),
        row=1, col=2)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['Cumulative Profit'], name='Total Profit', mode='lines', line=dict(color='green')),
        row=2, col=1)
    fig.add_trace(
        go.Scatter(x=df.index, y=df['E'], name='Battery Charge', mode='lines'),
        row=2, col=2)
    fig.add_hline(y=df['E'][0], line=dict(color='green', dash='dot'), row=2, col=2)
    # Update y-axis labels
    fig.update_yaxes(title_text="Price at time t", row=1, col=1)
    fig.update_yaxes(title_text="Charging rate (MWh)", row=1, col=2)
    fig.update_yaxes(title_text="Profit over time t", row=2, col=1)
    fig.update_yaxes(title_text="Battery charge at t", row=2, col=2)
    fig.update_layout(height=600, width=1500)
    fig.show()

def plot_profit(m_energy, color="g", label="Total profit"):
    df = storage_extract_singleindex(m_energy)
    
    fig = go.Figure()
    fig = fig.add_trace(go.Scatter(x=df['Cumulative Profit'].index, y=df['Cumulative Profit'],
                     name="Cumulative Profit", mode='lines'))
    fig.update_yaxes(title="Price")
    fig.show()


m_normal_forecast, res_normal_forecast = solve_system_deterministic(price, h_length=150)
plot_simple_model_deterministic(m_normal_forecast)
plot_profit(m_normal_forecast)

## Receding horizon controller (pyomo.mpc library)
- Converts Horizon to time series, forecast/nowcast data from LEOGO dataset
- Simulation steps is number solves of the RHC, and the model horizon is the length of the prediction horzion

The ```f_data``` and ```n_data``` can be expanded to most parameters, even the switching on/off of generators can be implemented
Different penalties that are mutable can be tested in this framework in a EMPC fashion, mut MVP is wind-data.

N = ```model_horizon```,  
S = ```simulation_steps``` ,  
H = ```measurement_horizon```,  
```noise_std``` = 0.06, for evaluation of more noise in the model and same parameter used in thesis

In [ ]:
# ----------------- Important simulation variables --------------------
model_horizon = 12 # prediction horizon
simulation_steps = 150
nowcast_horizon = 3         # Refreshing data ofter
noise_std = 0.06

## Simulate the actual data with gaussian noise

In [ ]:
std_dev = noise_std * price.std() 

# Generate Gaussian noise
noise = np.random.normal(0, std_dev, price.shape)

# Add the noise to the predictions to create a new 'Ideal data' column
noise_data = price + noise

fig_ideal = go.Figure()
fig_ideal.add_trace(go.Scatter(x=noise_data.index[:simulation_steps], y=noise_data[:simulation_steps], mode='lines', name="Ideal data"))

## EMPC implementation
- Gets a steady state
- Runs mpc with the parameters spesified

In [ ]:
def get_steady_state(target, tee=False):
        m = simple_model(price,horizon_length=model_horizon)
        interface = mpc.DynamicModelInterface(m, m.N)
        m.E0 = target
        m.vChargeRate[0].fix()
        m.vDischargeRate[0].fix()

        solver = pyo.SolverFactory("gurobi")
        solver.solve(m, tee=tee)
        return interface.get_data_at_time(0)

# Set initial steady state of the bess
initial_e0 = get_steady_state(target=2.0)

f_data = {
        "pPrice[*]": [x for x in price[:simulation_steps]],
}
n_data = {
        "pPrice[*]": [x for x in noise_data[:simulation_steps]]
}
m_perfect = simple_model(price, horizon_length=model_horizon) 
m_res_3, result_3 = run_rhc3(m_perfect, model_horizon, f_data, n_data, simulation_steps,
                                        initial_e0, nowcast_horizon=nowcast_horizon, verbose=False)

#### MPC simulation stops at simulation steps ```simulation_steps```
- ```Forecasted Profit``` is the model solved with the original forecast data, and the expected profit over the horizon, cumulated.
- ```Actual Profit``` is the model choices from the forecasted model, applied to the nosiy data, cumualted.
- ```MPC profit``` is the model profit expected when simulating and solving using nowcast data and forecast data

In [ ]:
# Apply the calculated forecast variables from the deterministic on the actual data
noise_data = noise_data[:simulation_steps]
# Extract mpc solution
df_res_3= pd.DataFrame(*result_3.to_serializable())
data = {   'Costs': noise_data* df_res_3['vChargeRate[*]'], 
        'Profit': noise_data*df_res_3['vDischargeRate[*]'] 
        }
df_res_3 = df_res_3.assign(**data)
df_res_3["Net profit"] = df_res_3["Profit"] - df_res_3["Costs"]
df3 = df_res_3["Net profit"].cumsum()

m_normal_forecast, _ = solve_system_deterministic(price, h_length=simulation_steps)
m_ideal, _ = solve_system_deterministic(noise_data, h_length=simulation_steps)

# extract normal forecast case
df_normal_forecast = storage_extract_singleindex(m_normal_forecast) 
df1 = df_normal_forecast["Cumulative Profit"]

# Extract ideal case
df_ideal = storage_extract_singleindex(m_ideal)
df2 = df_ideal["Cumulative Profit"]

fig_E = go.Figure()
fig_E.add_trace(go.Scatter(x=df_res_3["E[*]"].index, y=df_res_3["E[*]"], mode='lines', name="State of Charge MPC"))
fig_E.add_trace(go.Scatter(x=df_ideal["E"].index, y=df_ideal["E"], mode='lines', name="State of Charge ideal"))
fig_E.update_layout(title="BESS State of Charge", xaxis_title="Time", yaxis_title="MWh")
fig_cumsum = go.Figure()
fig_cumsum.add_trace(go.Scatter(x=df1.index, y=df1, mode='lines', name="Forecast"))
fig_cumsum.add_trace(go.Scatter(x=df2.index, y=df2, mode='lines', name="Actual"))
fig_cumsum.add_trace(go.Scatter(x=df3.index, y=df3, mode='lines', name="EMPC"))
fig_cumsum.update_layout(title="Cumulative Sum", xaxis_title="Time", yaxis_title="Profit")
fig_cumsum.show()